Software Platform Lab, Seoul National University

## Colab 101

Colab is a free Jupyter notebook environment by Google Research. Unlike AWS cluster (which is charged every hour it is up and running), you can run experiments on your own environment.

## Colab Spark Setup

In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless
!curl http://mirror.cogentco.com/pub/apache/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz --output spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 4s (63.7 kB/s)
Reading package lists... Done
Reading package lists... Done
Building d

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

## Wikipedia dataset sample

This time we're not using HDFS to load the data. Sample data are loaded by Python code directly.

The data has four fields: project, title, pageview count and size.

In [0]:
wikipedia_data_sample = ["commons.m File:Gemblong.JPG 1 9717"
,"pl Beata_Tyszkiewicz 10 207378"
,"en Special:RecentChangesLinked/Roswell_(TV_series) 1 14617"
,"de Grafische_Benutzeroberfl%C3%A4che 1 22549"
,"en Simeon_I_of_Bulgaria 5 385793"
,"en Rainbow_Six_(novel) 8 122792"
,"es Pediatr%C3%ADa 5 73598"
,"sv Ett_uts%C3%B6kt_universum 1 9499"
,"en Video_game_content_rating_system 4 112324"
,"es Yuno_Gasai 2 55260"
,"en File:Georg_Wilhelm_Friedrich_Hegel00.jpg 1 43395"
,"en Anestia_ombrophanes 1 8881"
,"et Seitse 2 84874"
,"en And_I_Am_Telling_You_I%27m_Not_Going 4 85690"
,"he %D7%A4%D7%A8%D7%93%D7%99%D7%92%D7%9E%D7%94 1 13887"
,"zh File:Pictogram_voting_keep-green.svg 1 15106"
,"sv Special:Senaste_relaterade_%C3%A4ndringar/Homestead,_Florida 1 7677"
,"pt Categoria:Ambientes_de_desenvolvimento_integrado_livres 1 8151"
,"de.voy Plattensee 1 43748"
,"en Independent_Chip_Model 1 8938"
,"en Category:Toronto_Toros_players 2 0"
,"en Special:Export/Helsinki_Accords 1 19899"
,"xh Special:Contributions/Kpeterzell 1 5883"
,"nl 4_mei 1 0"
,"no Carlos_Keller_Rueff 5 87075"
,"en Special:Contributions/2.31.218.202 1 7402"
,"es Placa_Yangtze 1 10329"
,"de Datei:BSicon_uhKBHFe.svg 1 9786"
,"en Randolph_County,_Alabama 1 21431"
,"es S%C3%A9neca 3 70494"
,"en Tu_Bishvat 3 56438"
,"cs Radiohead 1 14325"
,"es Naturaleza_sangre 1 9286"
,"en Anatolia_(disambiguation) 1 7980"
,"pt Queima_de_suti%C3%A3s 1 8982"
,"pt Titanoboa_cerrejonensis 5 64540"
,"commons.m Category:People_of_Ireland 1 19278"
,"fi Matti_Inkinen 1 10138"
,"ja %E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB:Esfahan_(Iran)_Emam_Mosque.JPG 1 33168"
,"en Psicobloc 1 12739"
,"en Macael,_Spain 1 12658"
,"fa %DA%A9%D9%87%D8%AA%D9%88%DB%8C%D9%87 1 22855"
,"fr Sp%C3%A9cial:Pages_li%C3%A9es/Fichier:Wiki-ezokuroten5.jpg 1 21955"
,"nl Overleg_gebruiker:82.171.157.232 1 0"
,"en Thomas_%26_Mack_Center 2 41010"
,"en Warren_Beatty 49 2631986"
,"uz Auberville 1 11401"]

## Spark RDD Transforms

Now we'll try several Spark RDD transforms using the sample wikipedia dataset.

In [0]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

ss = SparkSession.builder.master("local[*]").getOrCreate()
sc = ss.sparkContext

In [0]:
type(ss)

pyspark.sql.session.SparkSession

In [0]:
from pyspark.sql.functions import *
# Parallelize the data and split into columns
lines = sc.parallelize(wikipedia_data_sample)
columns = lines.map(lambda line: ((explode(line.split(" ")))))

In [0]:
lines.take(5)

['commons.m File:Gemblong.JPG 1 9717',
 'pl Beata_Tyszkiewicz 10 207378',
 'en Special:RecentChangesLinked/Roswell_(TV_series) 1 14617',
 'de Grafische_Benutzeroberfl%C3%A4che 1 22549',
 'en Simeon_I_of_Bulgaria 5 385793']

In [0]:
columns.take(5)

Py4JJavaError: ignored

In [0]:
# Element-Wise Transformation: Map Transform

# Create (project, count) tuples(Be mindful of 'long()'!)
project_count_tuples = columns.map(lambda column: (column[0], long(column[2])))
project_count_tuples.collect()

[('commons.m', 1L),
 ('pl', 10L),
 ('en', 1L),
 ('de', 1L),
 ('en', 5L),
 ('en', 8L),
 ('es', 5L),
 ('sv', 1L),
 ('en', 4L),
 ('es', 2L),
 ('en', 1L),
 ('en', 1L),
 ('et', 2L),
 ('en', 4L),
 ('he', 1L),
 ('zh', 1L),
 ('sv', 1L),
 ('pt', 1L),
 ('de.voy', 1L),
 ('en', 1L),
 ('en', 2L),
 ('en', 1L),
 ('xh', 1L),
 ('nl', 1L),
 ('no', 5L),
 ('en', 1L),
 ('es', 1L),
 ('de', 1L),
 ('en', 1L),
 ('es', 3L),
 ('en', 3L),
 ('cs', 1L),
 ('es', 1L),
 ('en', 1L),
 ('pt', 1L),
 ('pt', 5L),
 ('commons.m', 1L),
 ('fi', 1L),
 ('ja', 1L),
 ('en', 1L),
 ('en', 1L),
 ('fa', 1L),
 ('fr', 1L),
 ('nl', 1L),
 ('en', 2L),
 ('en', 49L),
 ('uz', 1L)]

In [0]:
# Element-Wise Transformation: Filter Transform

# Filter project containing name 'de'
project_de_filtered = project_count_tuples.filter(lambda t: 'de' in t[0])
# project_de_filtered.collect()

## Quiz 1
Sample wikipedia data에서 project 의 count column 값이 5 보다 큰 경우를 filter 하시오.
- 결과값: project, count 로 구성된 tuple

In [0]:
# FIXME
project_count_tuples.filter(lambda x : x[1]>5L).collect()

[('pl', 10L), ('en', 8L), ('en', 49L)]

In [0]:
# Transformations on one Pair RDD: ReduceByKey Transform

# Compute the sum of pageview counts per project
project_sum_tuples = project_count_tuples.reduceByKey(lambda left, right: left + right) 
filtered_tuple = project_sum_tuples.filter(lambda r : r[1] > 5L)
project_sum_tuples.collect()
print('filtered date')
filtered_tuple.collect()

filtered date


[('pt', 7L), ('es', 12L), ('pl', 10L), ('en', 87L)]

In [0]:
# Transformations on two Pair RDDs: Join Transform

# Declare another two sample data
wikipedia_sample_singer = ["en Steve_Jobs 49 2631986"
,"en WoodKid 1 12739"
,"en Honne 100 12658"
,"fa Eminem 1 22855"
,"en Sia 49 2631986"]

singer_to_ranking = ["WoodKid 1"
,"Honne 2"
,"Eminem 3"
,"Sia 4"]

# Parallelize the data and split into columns
lines2 = sc.parallelize(wikipedia_sample_singer)
lines3 = sc.parallelize(singer_to_ranking)

wikipedia_sample_singer_tuples = lines2.map(lambda line: tuple(line.split(" ")))
singer_to_ranking_tuples = lines3.map(lambda line: tuple(line.split(" ")))

# Create (title, count) tuples and join via title name.
title_count_tuples = wikipedia_sample_singer_tuples.map(lambda column: (column[1], long(column[2])))
title_count_tuples.join(singer_to_ranking_tuples).collect()

[('Honne', (100L, '2')),
 ('WoodKid', (1L, '1')),
 ('Sia', (49L, '4')),
 ('Eminem', (1L, '3'))]

## SparkSQL

In [0]:
# Create a Spark DataFrame from wikipedia_data_sample (equivalent of a 'SQL table' in Spark)
df = ss.createDataFrame(columns, ['project', 'title', 'count', 'size'])

# Create a table view called "WikipediaTable"
df.createOrReplaceTempView("WikipediaTable")

# Run an SQL query that selects project equals to 'en' with count greater than 5
selected = ss.sql("SELECT project, count FROM WikipediaTable \
                   WHERE project='en' AND count >= 5")

# Print the results in this console (top 20 results will be shown)
selected.show()

+-------+-----+
|project|count|
+-------+-----+
|     en|    5|
|     en|    8|
|     en|   49|
+-------+-----+



In [0]:
# Run an SQL query that orders projects by the number of titles each project has
selected = ss.sql("SELECT project, COUNT(title) as num_of_title FROM WikipediaTable \
                  GROUP BY project \
                  ORDER BY num_of_title DESC")

# Print the results in this console (top 20 results will be shown)
selected.show()

+---------+------------+
|  project|num_of_title|
+---------+------------+
|       en|          18|
|       es|           5|
|       pt|           3|
|       nl|           2|
|       de|           2|
|commons.m|           2|
|       sv|           2|
|       uz|           1|
|       cs|           1|
|       pl|           1|
|   de.voy|           1|
|       xh|           1|
|       no|           1|
|       zh|           1|
|       et|           1|
|       ja|           1|
|       he|           1|
|       fa|           1|
|       fr|           1|
|       fi|           1|
+---------+------------+



In [0]:
# Create a Spark DataFrame from singer_to_ranking and wikipedia_sample_singer
df = ss.createDataFrame(singer_to_ranking_tuples, ['title', 'ranking'])
df1 = ss.createDataFrame(wikipedia_sample_singer_tuples, ['project', 'title', 'count', 'size'])

# Create a table view of them, called "RankingTable" and "SingerTable"
df.createOrReplaceTempView("RankingTable")
df1.createOrReplaceTempView("SingerTable")

# Run an SQL query that joins the two tables.
# The result will show 'ranking' of RankingTable and 'title', 'count' of SingerTable.
# Join will be performed on rows with common 'title' in both tables.
selected = ss.sql("SELECT RankingTable.ranking, SingerTable.title, SingerTable.count FROM SingerTable \
                   INNER JOIN RankingTable ON RankingTable.title=SingerTable.title \
                   ORDER BY RankingTable.ranking")

selected.show()

+-------+-------+-----+
|ranking|  title|count|
+-------+-------+-----+
|      1|WoodKid|    1|
|      2|  Honne|  100|
|      3| Eminem|    1|
|      4|    Sia|   49|
+-------+-------+-----+



## Quiz 2. 
'WikipediaTable'에서, 각 project 당 count column 값의 총합이 20 이상인 (project, sum_of_count)를 구하시오
- 결과값: project, sum_of_count 2개의 column 을 갖는 테이블

In [0]:
import pyspark.sql.functions as sf
from pyspark.sql.types import StructType,StructField

In [0]:
# Create a Spark DataFrame from wikipedia_data_sample (equivalent of a 'SQL table' in Spark)
wkdf = ss.createDataFrame(columns, ['project', 'title', 'count', 'size'])

# Create a table view called "WikipediaTable"
wkdf.createOrReplaceTempView("Wiki")

# Run an SQL query that selects project equals to 'en' with count greater than 5
selected = ss.sql("SELECT project, sum(count) FROM Wiki \
                   group by project \
                   having sum(count)>=20")

# Print the results in this console (top 20 results will be shown)
selected.show()

+-------+--------------------------+
|project|sum(CAST(count AS DOUBLE))|
+-------+--------------------------+
|     en|                      87.0|
+-------+--------------------------+



In [0]:
from pyspark.sql.types import IntegerType,StringType,ArrayType,LongType
from pyspark.sql.functions import max,sum

In [0]:
agg_df = wkdf.withColumn('cnt',wkdf['count'].cast(IntegerType())).groupBy('project').agg(sum('cnt').alias('sumofcnt'))
agg_df.where(agg_df['sumofcnt']>50).show()

+-------+--------+
|project|sumofcnt|
+-------+--------+
|     en|      87|
+-------+--------+



In [0]:
## FIXME
# Create a Spark DataFrame from wikipedia_data_sample (equivalent of a 'SQL table' in Spark)


wkdf = ss.createDataFrame(columns, ['project', 'title', 'count', 'size'])
wkdf.head(5)
wkdf.printSchema()
# wkdf.groupBy('project').agg(sum('count')).head(10)
# Create a table view called "WikipediaTable"
# wkdf.createOrReplaceTempView("WikTbl")

# # Run an SQL query that selects project equals to 'en' with count greater than 5
# wkselected = ss.sql("SELECT project, count FROM WikiTbl \
#                    WHERE project='en' AND count >= 5")

# # Print the results in this console (top 20 results will be shown)
# wkselected.show()

root
 |-- project: string (nullable = true)
 |-- title: string (nullable = true)
 |-- count: string (nullable = true)
 |-- size: string (nullable = true)



## Quiz 3.
다음의 table을 'WikipediaTable'과 Join하여, grade가 'C'에 해당하는 project에 속하는 title들을 구하시오
- 결과값: title 1개의 column 을 갖는 테이블

In [0]:
cols = ['project', 'grade']
vals = [
     ('en', 'C'),
     ('he', 'A'),
     ('zh', 'B'),    
     ('no', 'A')
]

title_grade = ss.createDataFrame(vals, cols)
title_grade.show()
title_grade.createOrReplaceTempView("TitleGradeTable")

+-------+-----+
|project|grade|
+-------+-----+
|     en|    C|
|     he|    A|
|     zh|    B|
|     no|    A|
+-------+-----+



In [0]:
## FIXME
ss.sql("select wk.title,tg.grade from Wiki as wk \
        inner join TitleGradeTable as tg \
        on wk.project = tg.project \
        where tg.grade='C'").show()

+--------------------+-----+
|               title|grade|
+--------------------+-----+
|Special:RecentCha...|    C|
|Simeon_I_of_Bulgaria|    C|
| Rainbow_Six_(novel)|    C|
|Video_game_conten...|    C|
|File:Georg_Wilhel...|    C|
| Anestia_ombrophanes|    C|
|And_I_Am_Telling_...|    C|
|Independent_Chip_...|    C|
|Category:Toronto_...|    C|
|Special:Export/He...|    C|
|Special:Contribut...|    C|
|Randolph_County,_...|    C|
|          Tu_Bishvat|    C|
|Anatolia_(disambi...|    C|
|           Psicobloc|    C|
|       Macael,_Spain|    C|
|Thomas_%26_Mack_C...|    C|
|       Warren_Beatty|    C|
+--------------------+-----+

